In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/Data/

/content/drive/MyDrive/Colab Notebooks/Data


In [3]:
%cd toxic-comment/

/content/drive/MyDrive/Colab Notebooks/Data/toxic-comment


In [4]:
import pandas as pd
import numpy as np

In [5]:
df_train = pd.read_csv('train.csv')
df_test_file = pd.read_csv('test.csv')
df_test_labels = pd.read_csv('test_labels.csv')

In [6]:
df_train.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [7]:
train_labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [8]:
print(f"{(df_train[train_labels].sum()/df_train.shape[0]) * 100}" )

toxic            9.584448
severe_toxic     0.999555
obscene          5.294822
threat           0.299553
insult           4.936361
identity_hate    0.880486
dtype: float64


In [9]:
df = df_train.sample(20000)

In [10]:
print(f"{(df[train_labels].sum()/df.shape[0]) * 100}" )

toxic            9.475
severe_toxic     0.955
obscene          5.315
threat           0.255
insult           4.880
identity_hate    0.935
dtype: float64


In [11]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.1)

In [12]:
train_df.shape, val_df.shape

((18000, 8), (2000, 8))

In [13]:
train_df= train_df.drop('id', axis=1)

In [14]:
eval_df = val_df.drop('id', axis=1)

In [15]:
train_df.comment_text.iloc[1], train_df[train_labels].iloc[1]

('"\nIt would be interesting, Mr. 107, to complain of socks, given that your only edit to Wikipedia was to this article. Belt "',
 toxic            0
 severe_toxic     0
 obscene          0
 threat           0
 insult           0
 identity_hate    0
 Name: 9822, dtype: int64)

In [16]:
list(train_df[train_labels].iloc[1].values)

[0, 0, 0, 0, 0, 0]

In [17]:
def createDataset(data):
  new_data ={'comment_text':[], 'labels':[]}
  for i in range(data.shape[0]):
    text = data.comment_text.iloc[i]
    labels = list(data[train_labels].iloc[i].values)
    
    new_data['comment_text'].append(text)
    new_data['labels'].append(labels)

  new_dataframe = pd.DataFrame(new_data)
  return new_dataframe

In [18]:
train_dataset = createDataset(train_df)

In [ ]:
train_dataset['labels'].sum()

In [20]:
val_df.shape

(2000, 8)

In [21]:
eval_data = createDataset(val_df)

In [22]:
eval_data

,comment_text,labels
0,Spellcast Countercasted \n\nAs I told Spellcas...,"[0, 0, 0, 0, 0, 0]"
1,Please stop removing the info in the lead \n\n...,"[0, 0, 0, 0, 0, 0]"
2,"I'm angry now, I'm not spewing out abuse at yo...","[0, 0, 0, 0, 0, 0]"
3,Comments and concerns from Utahredrock,"[0, 0, 0, 0, 0, 0]"
4,Punk Music is huge in Boise as well as many pu...,"[0, 0, 0, 0, 0, 0]"
...,...,...
1995,Franklin's Saying Arranged By Subject \n\nSond...,"[0, 0, 0, 0, 0, 0]"
1996,""":::::I think this is a brilliant and wholly a...","[0, 0, 0, 0, 0, 0]"
1997,CAIR\nI noticed your comment on another user's...,"[0, 0, 0, 0, 0, 0]"
1998,"""\n\n Downgrade of USA's Credit Rating by Stan...","[0, 0, 0, 0, 0, 0]"


In [1]:
!pip install simpletransformers

In [23]:
from simpletransformers.classification import (
    MultiLabelClassificationModel, MultiLabelClassificationArgs
)
import pandas as pd
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [24]:
# Optional model configuration
model_args = MultiLabelClassificationArgs(num_train_epochs=5)

In [25]:
# Create a MultiLabelClassificationModel
model = MultiLabelClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=6,
    args=model_args,
)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultiLabelSequenceClassification: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'c

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [28]:
# Train the model
model.train_model(train_dataset, eval_df=eval_data)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/18000 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_roberta_128_0_2


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/2250 [00:00<?, ?it/s]

...

Running Epoch 2 of 5:   0%|          | 0/2250 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/2250 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/2250 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.


(11250, 0.04712865349036228)

In [29]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(
    eval_data
)



/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/2000 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_0_2


Running Evaluation:   0%|          | 0/250 [00:00<?, ?it/s]

In [30]:
print(result)
print(model_outputs)
print(wrong_predictions)

{'LRAP': 0.9925794444444447, 'eval_loss': 0.06832694689289201}
[[7.29560852e-04 1.79529190e-04 4.02927399e-04 1.25288963e-04
  3.61204147e-04 2.30550766e-04]
 [6.69479370e-04 1.97172165e-04 4.06026840e-04 1.36613846e-04
  3.68356705e-04 2.49147415e-04]
 [7.57598877e-03 9.24468040e-05 9.96589661e-04 9.38773155e-05
  7.52449036e-04 2.23398209e-04]
 ...
 [8.93592834e-04 1.51157379e-04 4.17232513e-04 1.07228756e-04
  3.51428986e-04 2.01821327e-04]
 [6.11782074e-04 2.21729279e-04 4.18901443e-04 1.48892403e-04
  3.80039215e-04 2.71558762e-04]
 [5.93185425e-04 2.32338905e-04 4.23908234e-04 1.54733658e-04
  3.84569168e-04 2.82526016e-04]]
[]


In [31]:
# Make predictions with the model
predictions, raw_outputs = model.predict(train_df.comment_text.iloc[1])

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/124 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [33]:
print(predictions)
#print(raw_outputs)

[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0],

In [34]:
predictions, raw_outputs = model.predict(["You  are a bad person"])

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [35]:
predictions

[[1, 0, 0, 0, 1, 0]]

In [36]:
predictions, raw_outputs = model.predict(["Stars are Shining"])
predictions

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[[0, 0, 0, 0, 0, 0]]

In [41]:
predictions, raw_outputs = model.predict(["you are pathetic"])
predictions

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[[1, 0, 0, 0, 1, 0]]